In [1]:
import os
import json
import pandas as pd
import fnmatch
# from pyannote.audio import Pipeline
from tqdm import tqdm

In [13]:
data_path = './ABI_data/Audios/'
path = './ABI_data/'
diary_path = './ABI_data/diarization/'

with open(path+'selected.txt', 'r') as f:
    lines = f.readlines()
    series = []
    for line in lines:
        series.append(line.strip('\n'))
series

['Brain Injury Today',
 'Life After Brain Injury by Will and Amy Perringwood',
 'Expressions of Life With an Acquired Brain Injury',
 'Between Two Brains by the Brain Injury Association',
 'Brainbank by Synapse',
 'Brain Injury Conversations by Cynthia Burke']

In [ ]:
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization")

In [ ]:
data_path = './ABI_data/Audios/'
diary_path = data_path + 'diarization/'
if not os.path.exists(diary_path):
    os.mkdir(diary_path)
# series = sorted(os.listdir(data_path))

for podcast in series:
    files = fnmatch.filter(os.listdir(data_path+podcast), '*.wav')
    files.sort()
    output_path = diary_path + podcast + '/'
    if not os.path.exists(output_path):
        os.mkdir(output_path)
    
    print('Processing %s' % podcast)
    count = 1
    for file in tqdm(files):
        inst = file.split('.')[0]
        diarization = pipeline(data_path+podcast+'/'+file)
        diary = dict()
        
        for turn, _, speaker in diarization.itertracks(yield_label=True):
            
            detail = dict()
            detail['episode'] = inst
            detail['start'] = turn.start
            detail['end'] = turn.end
            detail['speaker'] = speaker

            key = podcast + '_' + str(count).zfill(5)
            diary[key] = detail

            count += 1
            
#             print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")
        with open(output_path+inst+'.json', "w") as outfile:
            json.dump(diary, outfile, indent=4)
    
    

In [15]:
diary_path

'./ABI_data/diarization/'

In [19]:
podcasts = os.listdir(diary_path)
podcasts.sort()
min_time = 1.0

for podcast in series:
    output_path = diary_path + podcast + '/'
    diaries = os.listdir(output_path)
    diaries.sort()
    count = 1
    
    for d in diaries:
        if d in '_cleaned':
            continue
        
        f = open(diary_path + podcast + '/' + d)
        inst = d.split('.')[0]
        data = json.load(f)
        last_speaker = " "
        start_offset = 0

        diary = dict()

        for key in data:
            segment = data[key]

            if last_speaker == " ":
                last_speaker = segment['speaker']
                start_offset = segment['start']
                utter_end = segment['end']
                continue
            if segment['speaker'] != last_speaker or (segment['speaker'] == last_speaker and segment['start'] - utter_end > 2.0):
                detail = dict()
                detail['episode'] = segment['episode']
                detail['start'] = start_offset
                detail['end'] = utter_end
                detail['speaker'] = last_speaker

                new_key = key.split('_')[0] + '_'+str(count).zfill(5)
                diary[new_key]  = detail

                count += 1
                last_speaker = segment['speaker']
                start_offset = segment['start']
            utter_end = segment['end']
    
        with open(output_path+inst+'_cleaned.json', "w") as outfile:
            json.dump(diary, outfile, indent=4)
    print(podcast + 'has' + str(count))

Brain Injury Todayhas1899
Life After Brain Injury by Will and Amy Perringwoodhas910
Expressions of Life With an Acquired Brain Injuryhas84
Between Two Brains by the Brain Injury Associationhas1085
Brainbank by Synapsehas297
Brain Injury Conversations by Cynthia Burkehas2215


In [18]:
for podcast in series:
    diaries = os.listdir(diary_path + podcast + '/')
    max_duration = 0
    
    for diary in diaries:
        f = open(diary_path + podcast + '/' + diary)
        j = json.load(f)
        for key in j:
            segment = j[key]
            start = segment['start']
            end = segment['end']
            duration = end - start
            if duration > max_duration:
                max_duration = duration
                max_episode = segment['episode']
                max_key = key
      
    print(podcast + ' has %f s max_duration' % max_duration)

Brain Injury Today has 332.859375 s max_duration
Life After Brain Injury by Will and Amy Perringwood has 119.761875 s max_duration
Expressions of Life With an Acquired Brain Injury has 351.607500 s max_duration
Between Two Brains by the Brain Injury Association has 286.335000 s max_duration
Brainbank by Synapse has 412.948125 s max_duration
Brain Injury Conversations by Cynthia Burke has 162.860625 s max_duration


In [12]:
print('_clean' in d+'_clean')

True
